In [ ]:
import sys
sys.path.append('..')

from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime

from data.twins.data import TwinsData


from algorithm.general import PearsonConfounderTest, GConfounderTest
from algorithm.kernel.test import KernelConfounderTest
from algorithm.misc import PearsonPrognosticConfounderTest, KernelPrognosticConfounderTest


from experiment.plot import set_mpl_default_settings, marker_dict, color_dict
set_mpl_default_settings()

In [ ]:
from econml.dml import LinearDML
from sklearn.linear_model import LinearRegression

def estimate_ate(data, covariates):
    T = data['T']
    Y = data['Y']
    if len(covariates) > 0:
        W = data[covariates]  
        est = LinearDML(model_y=LinearRegression(), model_t=LinearRegression(), discrete_treatment=False)
        est.fit(Y, T, W=W)
        ate = est.effect(T0=0, T1=1)[0]
    else:
        est = LinearRegression()
        est.fit(T.values.reshape(-1,1), Y.values)
        ate = est.coef_[0]
    return ate

In [ ]:
def experiment(method, conf_strength, alpha, nbr_iter, resample_mechnisms, nbr_confounders=None, nbr_observed_confounders=0, compute_bias=True, max_tests=None):
    
    data_gen = TwinsData('../data/twins/files', nbr_confounders=nbr_confounders)
    conf_rate = 0
    bias  = []
    statistic = []
    for _ in range(nbr_iter):
        
        # sample data
        data_gen.randomly_select_confounders()
        confounder_list = data_gen.covar_list
        out = data_gen.sample(conf_strength, resample_mechanisms=resample_mechnisms, binary=False, nbr_changes=0)
        data = out['observational_data']
        
        # choose nbr_observed_confounders from confounder_list
        observed_covariates = list(np.random.choice(confounder_list, size=nbr_observed_confounders, replace=False))
        
        # run test
        res = method.test(data, observed_covariates=observed_covariates, max_tests=max_tests, alpha=alpha)
        pval = res['pval']
        
        if type(method).__name__ in ['PearsonConfounderTest', 'KernelConfounderTest', 'KernelPrognosticConfounderTest', 'PearsonPrognosticConfounderTest']:
            statistic.append(res['X'])
        else:
            statistic.append(0)

        if pval < alpha:
            conf_rate += 1

        # compute bias of ate
        if compute_bias:
            for e in out['true_ate']:
                true_ate = out['true_ate'][e]
                est_ate = estimate_ate(data[e], observed_covariates)
                bias.append(np.abs(est_ate - true_ate))
        else:
            for e in out['true_ate']:
                bias.append(-1)
    
    return conf_rate/nbr_iter, np.mean(bias), np.std(bias), np.mean(statistic), np.std(statistic), res['threshold']

## Experiment
For each method, we vary
- confounding strength
- number of observed confounders
- nbr of changes (sparsity)

In [ ]:
load = False

# Experiment parameters
methods = [KernelConfounderTest()]
conf_strengths = [0, 0.25, 1, 2.5, 5] 
nbr_observed_confounders = [0,1,2,3,4,5]
max_tests = [1,5,50]
alpha = 0.05
nbr_iter = 50
nbr_confounders = [3,5]
resample_mechanisms = True
compute_bias = True


# Get timestamp for experiment
now = datetime.now()
timestamp = now.strftime("%m%d%H%M")
print('Timestamp:', timestamp)

In [ ]:
if not load:

    res = {'detection_rate' : [], 
            'bias_mean' : [], 
            'bias_std' : [],
            'method' : [],
            'conf_strength' : [],
            'nbr_observed_confounders' : [],
            'max_tests' : [],
            'resample_mechanisms' : [],
            'alpha' : [],
            'nbr_iter' : [],
            'stat_mean' : [],
            'stat_std' : [],
            'threshold' : [],
            'nbr_confounders' : []
            }

    for i, method in enumerate(methods):
        for j, conf_strength in tqdm(enumerate(conf_strengths)):
            for k, n_obs_conf in enumerate(nbr_observed_confounders):
                    for m, max_test in enumerate(max_tests):
                        for n, nbr_conf in enumerate(nbr_confounders):
                            if nbr_conf < n_obs_conf:
                                continue
                            print('Method:', type(method).__name__, 'Confounder strength:', conf_strength, 'Nbr observed confounders:', n_obs_conf, 'Max tests:', max_test, 'Nbr confounders:', nbr_conf)
                            conf_rate, bias_mean, bias_std, stat_mean, stat_std, threshold = experiment(method=method, 
                                                                                                        conf_strength=conf_strength, 
                                                                                                        alpha=alpha, 
                                                                                                        nbr_iter=nbr_iter, 
                                                                                                        resample_mechnisms=resample_mechanisms,
                                                                                                        nbr_confounders=nbr_conf, 
                                                                                                        nbr_observed_confounders=n_obs_conf,
                                                                                                        compute_bias=compute_bias,
                                                                                                        max_tests=max_test)
                            res['detection_rate'].append(conf_rate)
                            res['bias_mean'].append(bias_mean)
                            res['bias_std'].append(bias_std)
                            res['method'].append(type(method).__name__)
                            res['conf_strength'].append(conf_strength)
                            res['nbr_observed_confounders'].append(n_obs_conf)
                            res['max_tests'].append(max_test)
                            res['resample_mechanisms'].append(resample_mechanisms)
                            res['alpha'].append(alpha)
                            res['nbr_iter'].append(nbr_iter)
                            res['stat_mean'].append(stat_mean)
                            res['stat_std'].append(stat_std)
                            res['threshold'].append(threshold)
                            res['nbr_confounders'].append(nbr_conf)
                    
    # Save res as DataFrame and into csv file
    df = pd.DataFrame(res)
    df.to_csv(f'results/twins_{timestamp}.csv')

else:

    timestamp_str = "11011016"
    timestamp = int(timestamp_str)
    path = f'results/twins_{timestamp}.csv'
    df = pd.read_csv(path)

In [ ]:
df.describe()

### Plot the variation of confounding strength versus detection rate

In [ ]:
fix_nbr_observed_confounders = 0
fix_max_tests = 50
fix_nbr_conf = 5
tmp_df = df[df['nbr_observed_confounders'] == fix_nbr_observed_confounders]
tmp_df = tmp_df[tmp_df['max_tests'] == fix_max_tests]
tmp_df = tmp_df[tmp_df['nbr_confounders'] == fix_nbr_conf]

plt.figure(figsize=(6,4))
for i, method in enumerate(tmp_df.method.unique()):
    plt.plot(tmp_df.conf_strength[tmp_df.method == method], tmp_df.detection_rate[tmp_df.method == method], label=method, marker=marker_dict[method], color=color_dict[method])
    lb = [p - np.sqrt(p*(1-p)/nbr_iter) for p in tmp_df.detection_rate[tmp_df.method == method].values]
    ub = [p + np.sqrt(p*(1-p)/nbr_iter) for p in tmp_df.detection_rate[tmp_df.method == method].values]
    plt.fill_between(tmp_df.conf_strength[tmp_df.method == method], lb, ub, alpha=0.2, color=color_dict[method])
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xlabel('Confounder strength')
plt.ylabel('Detection rate')
plt.title(f'Nbr observed confounders: {fix_nbr_observed_confounders}, Max tests: {fix_max_tests}')
plt.ylim(-.1,1.1)

plt.savefig(f'results/figures/twins_vary_conf_{timestamp}.pdf', bbox_inches='tight')

In [ ]:
plt.figure(figsize=(6,4))
for i, method in enumerate(tmp_df.method.unique()):
    plt.errorbar(tmp_df.bias_mean[tmp_df.method == method],
                    tmp_df.stat_mean[tmp_df.method == method], 
                    xerr=tmp_df.bias_std[tmp_df.method == method], 
                    yerr= tmp_df.stat_std[tmp_df.method == method], #[np.sqrt(p*(1-p)/nbr_iter) for p in tmp_df.detection_rate[tmp_df.method == method].values],
                    capsize=4,
                    capthick=2,
                    linestyle='--',
                    label=method, 
                    marker=marker_dict[method], 
                    color=color_dict[method])
plt.axhline(tmp_df.threshold.values[0], color='black', linestyle='-.', alpha=0.9)
plt.xlabel('Average omitted variable bias')
plt.ylabel('Average test statistic')
plt.savefig(f'results/figures/twins_vary_bias_{timestamp}.pdf', bbox_inches='tight')

### Plot the variation of nbr of observed confounders versus detection rate

In [ ]:
fix_conf_strength = 5
fix_max_tests = 50
fix_nbr_conf = 3
tmp_df = df[df['conf_strength'] == fix_conf_strength]
tmp_df = tmp_df[tmp_df['max_tests'] == fix_max_tests]


plt.figure(figsize=(6,4))
tmp_df = tmp_df[tmp_df['nbr_confounders'] == fix_nbr_conf]
for i, method in enumerate(tmp_df.method.unique()):
    plt.plot(tmp_df.nbr_observed_confounders[tmp_df.method == method], tmp_df.detection_rate[tmp_df.method == method], label=method, marker=marker_dict[method], color=color_dict[method])
    lb = [p - np.sqrt(p*(1-p)/nbr_iter) for p in tmp_df.detection_rate[tmp_df.method == method].values]
    ub = [p + np.sqrt(p*(1-p)/nbr_iter) for p in tmp_df.detection_rate[tmp_df.method == method].values]
    plt.fill_between(tmp_df.nbr_observed_confounders[tmp_df.method == method], lb, ub, alpha=0.2, color=color_dict[method])
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xlabel('Number of confounders adjusted for')
plt.ylabel('Detection rate')
plt.title(f'Conf strength: {fix_conf_strength}, Max tests: {fix_max_tests}')
plt.ylim(-.1,1.1)

plt.savefig(f'results/figures/twins_vary_obs_{timestamp}.pdf', bbox_inches='tight')

In [ ]:
plt.figure(figsize=(6,4))
for i, method in enumerate(tmp_df.method.unique()):
    plt.plot(tmp_df.nbr_observed_confounders[tmp_df.method == method], tmp_df.stat_mean[tmp_df.method == method], label=method, marker=marker_dict[method], color=color_dict[method])
    lb = tmp_df.stat_mean[tmp_df.method == method].values - tmp_df.stat_std[tmp_df.method == method].values
    ub = tmp_df.stat_mean[tmp_df.method == method].values + tmp_df.stat_std[tmp_df.method == method].values
    plt.fill_between(tmp_df.nbr_observed_confounders[tmp_df.method == method], lb, ub, alpha=0.2, color=color_dict[method])

plt.xlabel('Number of confounders adjusted for')
plt.ylabel('Average value of test statistic')
plt.title(f'Conf strength: {fix_conf_strength}, Max tests: {fix_max_tests}')
plt.axhline(tmp_df.threshold.values[0], color='black', linestyle='--', label='Reject at $\\alpha=0.05$')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
#plt.yscale('symlog')


In [ ]:
plt.figure(figsize=(6,4))
for i, method in enumerate(tmp_df.method.unique()):
    plt.plot(tmp_df.nbr_observed_confounders[tmp_df.method == method], tmp_df.bias_mean[tmp_df.method == method])
    lb = tmp_df.bias_mean[tmp_df.method == method].values - tmp_df.bias_std[tmp_df.method == method].values
    ub = tmp_df.bias_mean[tmp_df.method == method].values + tmp_df.bias_std[tmp_df.method == method].values
    plt.fill_between(tmp_df.nbr_observed_confounders[tmp_df.method == method], lb, ub, alpha=0.2, color=color_dict[method])
    break
plt.xlabel('Number of confounders adjusted for')
plt.ylabel('Omitted variable bias')
plt.ylim(0,10)

## Check how test depends on number of tests performed in Fisher's method


In [ ]:
fix_conf_strength = 5
fix_nbr_conf = 5
fix_max_tests_list = [1,5,50]
tmp_df = df[df['conf_strength'] == fix_conf_strength]
tmp_df = tmp_df[tmp_df['nbr_confounders'] == fix_nbr_conf]

plt.figure(figsize=(6,4))
for fix_max_tests in fix_max_tests_list:
    tmp2_df = tmp_df[tmp_df['max_tests'] == fix_max_tests]
    for i, method in enumerate(tmp_df.method.unique()):
        p = plt.plot(tmp2_df.nbr_observed_confounders[tmp_df.method == method], tmp2_df.stat_mean[tmp_df.method == method], label= method + f' $n_c=${fix_max_tests}', marker=marker_dict[method])
        plt.plot(tmp2_df.nbr_observed_confounders[tmp_df.method == method], tmp2_df.threshold[tmp_df.method == method], linestyle='--', color=p[-1].get_color())
        lb = tmp2_df.stat_mean[tmp_df.method == method].values - tmp2_df.stat_std[tmp_df.method == method].values
        ub = tmp2_df.stat_mean[tmp_df.method == method].values + tmp2_df.stat_std[tmp_df.method == method].values
        plt.fill_between(tmp2_df.nbr_observed_confounders[tmp_df.method == method], lb, ub, alpha=0.2)
    
        
plt.xlabel('Number of confounders adjusted for')
plt.ylabel('Average value of test statistic')
plt.title(f'Conf strength: {fix_conf_strength}, Max tests: {fix_max_tests}')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

In [ ]:
plt.figure(figsize=(6,4))
colors = ['green', 'blue', 'orange']
for i, fix_max_tests in enumerate(fix_max_tests_list):
    tmp2_df = tmp_df[tmp_df['max_tests'] == fix_max_tests]
    for j, method in enumerate(tmp_df.method.unique()):
        plt.plot(tmp2_df.nbr_observed_confounders[tmp_df.method == method],
                    tmp2_df.detection_rate[tmp_df.method == method],
                    label=f'$n_c=${fix_max_tests}',
                    marker=marker_dict[method],
                    color=colors[i])
        lb = [p - np.sqrt(p*(1-p)/nbr_iter) for p in tmp2_df.detection_rate[tmp_df.method == method].values]
        ub = [p + np.sqrt(p*(1-p)/nbr_iter) for p in tmp2_df.detection_rate[tmp_df.method == method].values]
        plt.fill_between(tmp2_df.nbr_observed_confounders[tmp_df.method == method], lb, ub, alpha=0.2, color=colors[i])
plt.legend()
plt.xlabel('Number of confounders adjusted for')
plt.ylabel('Detection rate')
plt.ylim(-.1,1.1)
plt.hlines(0.05, 0, tmp2_df.nbr_observed_confounders.max(), linestyle='--', color='black')
plt.savefig(f'./results/figures/twins_obs_conf_{timestamp}_conf{tmp2_df.nbr_observed_confounders.max()}.pdf', bbox_inches='tight')

# Investigate influence of adjusting for more confounders

In [ ]:
fix_conf_strength = 5
fix_max_tests = 50
nbr_confounder_list = [1,3,5]

tmp_df = df[df['conf_strength'] == fix_conf_strength]
tmp_df = tmp_df[tmp_df['max_tests'] == fix_max_tests]

plt.figure(figsize=(6,4))
for i, method in enumerate(tmp_df.method.unique()):
    tmp2_df = tmp_df[tmp_df['method'] == method]
    rate_list = []
    std_list = []
    for nbr_conf in nbr_confounder_list:
        tmp3_df = tmp2_df[tmp2_df['nbr_confounders'] == nbr_conf]
        tmp3_df = tmp3_df[tmp3_df['nbr_observed_confounders'] == nbr_conf]
        rate = tmp3_df.detection_rate.values[0]#tmp3_df.stat_mean.values[0]
        std = np.sqrt(rate*(1-rate)/nbr_iter)#tmp3_df.stat_std.values[0] 
        rate_list.append(rate)
        std_list.append(std)

    print(method, rate_list, std_list)

    p = plt.plot(nbr_confounder_list, rate_list, label= method + f' $n_c=${fix_max_tests}', marker=marker_dict[method])
    lb = [r - s for r,s in zip(rate_list, std_list)]
    ub = [r + s for r,s in zip(rate_list, std_list)]
    plt.fill_between(nbr_confounder_list, lb, ub, alpha=0.2)

    #plt.plot(nbr_confounder_list, tmp2_df.bias_mean, linestyle='--', color=p[-1].get_color())
    #lb = tmp2_df.bias_mean.values - tmp2_df.bias_std.values
    #ub = tmp2_df.bias_mean.values + tmp2_df.bias_std.values
    #plt.fill_between(nbr_confounder_list, lb, ub, alpha=0.2)

plt.legend(loc='upper right', bbox_to_anchor=(2.5, 1))
plt.xlabel('Number of confounders adjusted for')
plt.ylabel('Detection rate')
plt.title(f'Conf strength: {fix_conf_strength}, Max tests: {fix_max_tests}')